### Intended as a script which generates a .cvs report in one run

In [1]:
import tensorflow as tf;
import numpy as np;
import matplotlib.pyplot as plt;
print(tf.__version__);
import pandas as pd;

from sklearn.preprocessing import StandardScaler;
from sklearn.metrics import mean_squared_error;

2.3.0


In [2]:
pd.set_option('display.max_columns', 300);
pd.set_option('display.max_rows', 300);
tf.keras.backend.set_floatx('float64');

In [3]:
from tensorflow.python.client import device_lib

def get_available_gpus():
    local_device_protos = device_lib.list_local_devices()
    return [x.name for x in local_device_protos if x.device_type == 'GPU']

get_available_gpus()

[]

In [4]:
MONTHS = 72;
SPLIT = 60; # 2014-2018: training, 2019: testing.
# BATCH_SIZE = 19; # used in NN_v1
BATCH_SIZE = 30;
WINDOW_SIZE = 3;

TEST_LENGTH = MONTHS - SPLIT;

### Data preparation

In [5]:
multi_data = pd.read_csv('../data/zri_acs_bikeshare_merged.csv', index_col = 0);
zip_ids = multi_data.index.unique();

multi_data.drop(["City", "State", "Metro", "CountyName", "year", "month", "datetime"],\
                 axis = 1, inplace = True);

multi_data.head()

,zri,percent_white,percent_black,percent_asian,percent_hispanic,percent_native_am,percent_other_race,percent_0_17,percent_18_39,percent_40_64,percent_65+,percent_rental_units_vacant,percent_rental_units_occupied,percent_graduate_deg,percent_bachelors,percent_associates,percent_highschool,percent_less_highschool,percent_commute_public_transport,percent_commute_less_30,percent_buildings_less_10_units,percent_buildings_10_19_units,percent_buildings_20_49_units,percent_buildings_50+_units,percent_commute_30_to_59,percent_commute_60_to_89,percent_commute_90_more,percent_new_city,percent_new_unit,percent_units_owner_occupied,median_building_age,income_per_capita,poverty_rate,total_pop,percent_workforce_unemployed,percent_work_from_home,median_age,percent_female,gini_index,percent_not_us_citizen,bs_total_stations,bs_total_systems,has_bike_sharing
zip,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1013,928.0,0.728315,0.020951,0.013732,0.219830,0.0,0.004959,0.230010,0.311464,0.309663,0.148864,0.068131,0.931869,0.028046,0.056892,0.259500,0.384872,0.225671,0.023499,0.741732,0.890613,0.030177,0.034856,0.044355,0.227721,0.017344,0.013031,0.108543,0.040940,0.443651,62.0,20475.25,0.191440,22343.583333,0.104130,0.009346,36.416667,0.509533,0.420742,0.056062,0.0,0.0,0
1013,931.0,0.727052,0.021179,0.013708,0.220867,0.0,0.004917,0.229526,0.311552,0.310406,0.148516,0.067207,0.932793,0.028179,0.057377,0.259450,0.384586,0.225073,0.023365,0.740587,0.889943,0.030154,0.035083,0.044820,0.227757,0.017701,0.013401,0.108828,0.040926,0.441963,62.0,20517.50,0.192359,22296.166667,0.104147,0.009839,36.433333,0.508816,0.420483,0.056165,0.0,0.0,0
1013,934.0,0.725788,0.021408,0.013685,0.221904,0.0,0.004874,0.229041,0.311641,0.311150,0.148168,0.066283,0.933717,0.028313,0.057863,0.259400,0.384301,0.224475,0.023230,0.739441,0.889273,0.030131,0.035311,0.045285,0.227792,0.018058,0.013771,0.109112,0.040912,0.440275,62.0,20559.75,0.193277,22248.750000,0.104165,0.010332,36.450000,0.508100,0.420225,0.056267,0.0,0.0,0
1013,929.0,0.724525,0.021636,0.013661,0.222940,0.0,0.004832,0.228557,0.311729,0.311894,0.147820,0.065359,0.934641,0.028446,0.058348,0.259350,0.384016,0.223877,0.023096,0.738295,0.888603,0.030108,0.035538,0.045750,0.227828,0.018415,0.014142,0.109397,0.040897,0.438587,62.0,20602.00,0.194196,22201.333333,0.104182,0.010825,36.466667,0.507383,0.419967,0.056370,0.0,0.0,0
1013,929.0,0.723261,0.021864,0.013638,0.223977,0.0,0.004789,0.228072,0.311818,0.312638,0.147472,0.064435,0.935565,0.028579,0.058833,0.259299,0.383730,0.223279,0.022961,0.737149,0.887934,0.030086,0.035766,0.046215,0.227864,0.018772,0.014512,0.109682,0.040883,0.436898,62.0,20644.25,0.195114,22153.916667,0.104199,0.011318,36.483333,0.506666,0.419708,0.056472,0.0,0.0,0


In [6]:
FEATURES = multi_data.shape[1] - 1;

### Utility functions

In [7]:
@tf.autograph.experimental.do_not_convert
def windowed_dataset(series, window_size, batch_size, shuffle_buffer):
    dataset = tf.data.Dataset.from_tensor_slices(series); #(43,)
    dataset = dataset.window(window_size + 1, shift=1, drop_remainder=True);
    dataset = dataset.flat_map(lambda window: window.batch(window_size + 1)); #(13,43)
    dataset = dataset.shuffle(shuffle_buffer)\
                     .map(lambda window: (window[:-1], window[-1][0]));
    dataset = dataset.batch(batch_size).prefetch(1);
    return dataset;

In [8]:
def plot_series(time, series, format="-", start=0, end=None):
    plt.plot(time[start:end], series[start:end], format)
    plt.xlabel("Time Frame")
    plt.ylabel("ZRI")
    plt.grid(True)

### Neural network center

In [9]:
class myCallbacks(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs={}):
        mse = logs.get("mse");
        if(mse < 0.0025):
            print("\nGot an mse at {:.4f} in round {} and stopped training\n".format(mse, epoch));
            self.model.stop_training = True;

In [10]:
def NN_model(dataset, test = None):
    tf.keras.backend.clear_session();
    # dataset = windowed_dataset(x_train, window_size, batch_size, shuffle_buffer_size)

    callback = myCallbacks();
    
    model = tf.keras.models.Sequential([
#         tf.keras.layers.Lambda(lambda x: tf.expand_dims(x, axis=-1),
#                           input_shape=[None]),
#       tf.keras.layers.Conv1D(filters=32, kernel_size=3,
#                           strides=1, padding="causal",
#                           activation="relu",
#                           input_shape=[None, WINDOW_SIZE, FEATURES+1]),
        tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(16, return_sequences=True, input_shape = [None, WINDOW_SIZE, FEATURES+1])),
        #tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(16, return_sequences=True)),
        # tf.keras.layers.SimpleRNN(8, return_sequences=True),
        #  tf.keras.layers.SimpleRNN(16, return_sequences=True),
        tf.keras.layers.Dense(8, activation="relu"),
    #   tf.keras.layers.Dense(16, activation="relu"),
        tf.keras.layers.Dense(1)
        # tf.keras.layers.Lambda(lambda x: x * 2.0)
    ]);
    
    optimizer = tf.keras.optimizers.SGD(learning_rate=3e-4, momentum=0.9)
    model.compile(loss=tf.keras.losses.Huber(),
                  optimizer="adam",
                  metrics=["mae", "mse"])

    model.build((None,12,43))
    # model.summary()

    if not test: history = model.fit(dataset, epochs=500, callbacks = [callback], verbose = 0);
    else: history = model.fit(dataset, epochs=500, validation_data=test,\
                              callbacks=[callback], verbose = 0);
    return model;

In [11]:
def NN_forecast(model, series_transformed):
    forecast = []
    results = []
    for time in range(MONTHS - WINDOW_SIZE):
        forecast.append(model.predict(series_transformed[np.newaxis, time:time + WINDOW_SIZE, :]))

    results = [float(x[-1][0]) for x in forecast];
    timeline = range(WINDOW_SIZE, MONTHS);
    time_test = range(SPLIT, MONTHS);
    actual = list(series_transformed[WINDOW_SIZE:, 0]);

    forecast = series_transformed[SPLIT - WINDOW_SIZE:,:].copy();

    for time in range(TEST_LENGTH): # Change temp[time + WINDOW_SIZE]
        forecast[time + WINDOW_SIZE, 0] =\
        model.predict(forecast[np.newaxis, time:time + WINDOW_SIZE, :])[-1][0];

    pure_forecast = forecast[WINDOW_SIZE:,0];
    
    print(len(results), len(actual), len(pure_forecast))
    
    return results, actual, pure_forecast;

In [12]:
@tf.autograph.experimental.do_not_convert
def NN_test(ZONE, plot=False):
    '''
    Input: ZONE
    Output: the RMSE of a NN model on the predicted train, partially predicted test, and complete predicted test.
    '''
    # Collection of data
    series_frame = multi_data[multi_data.index == ZONE];
    # series_frame.sort_values("datetime", ascending = True, inplace = True);
    
    # Standardization
    scaler = StandardScaler();
    series_transformed = scaler.fit_transform(series_frame);
    
    # Train test split
    series_train = series_transformed[:SPLIT];
    series_test = series_transformed[SPLIT-WINDOW_SIZE:];
    
    # Window the training set to make input of the NN
    dataset_train = windowed_dataset(series_train, WINDOW_SIZE, BATCH_SIZE, 60);
    # dataset_test = windowed_dataset(series_test, WINDOW_SIZE, BATCH_SIZE, 60);
    
    model = NN_model(dataset_train);
    
    time_train = list(range(SPLIT));
    time_test = list(range(SPLIT, MONTHS));
    
    # Forecasting
    results, actual, pure_forecast = NN_forecast(model, series_transformed);
    
    # Compute MSE
    MSE_train = mean_squared_error(actual[:-TEST_LENGTH], results[:-TEST_LENGTH])**0.5 * scaler.var_[0]**0.5;
    MSE_test = mean_squared_error(actual[-TEST_LENGTH:], results[-TEST_LENGTH:])**0.5 * scaler.var_[0]**0.5;
    MSE_pure = mean_squared_error(actual[-TEST_LENGTH:], pure_forecast[-TEST_LENGTH:])**0.5 * scaler.var_[0]**0.5;
    
    if plot: # If the plot option is selected, plot the graph.
        time_actual = range(WINDOW_SIZE, MONTHS);
        plt.figure(figsize=(10, 6))
        plot_series(time_actual, actual);
        plot_series(time_actual, results);
        plot_series(time_test, pure_forecast);
        plt.show();

    return MSE_train, MSE_test, MSE_pure,\
        np.array(pure_forecast[-TEST_LENGTH:]) * scaler.var_[0]**0.5 + scaler.mean_[0];

### The script

In [13]:
score_dict = {"zip":[], "RMSE_train":[], "RMSE_test":[], "RMSE_pure":[], "forecast":[]};
with tf.device('/cpu:0'):
    for ZONE in zip_ids:
        %time M_train, M_test, M_pure, forecast = NN_test(ZONE, plot=False);
        print(ZONE, M_train, M_test, M_pure, forecast);

        score_dict["zip"].append(ZONE);
        score_dict["RMSE_train"].append(M_train);
        score_dict["RMSE_test"].append(M_test);
        score_dict["RMSE_pure"].append(M_pure);
        score_dict["forecast"].append(forecast);


Got an mse at 0.0025 in round 373 and stopped training

69 69 12
Wall time: 15.5 s
1013 3.6753699876156354 67.65090189667852 68.55885758039157 [1094.24625597 1088.93824278 1084.50145412 1080.91993171 1077.87497652
 1074.88865255 1071.96933294 1069.10025718 1066.25259239 1063.41327409
 1060.56565776 1057.69479148]

Got an mse at 0.0025 in round 228 and stopped training

69 69 12
Wall time: 11.4 s
1020 3.737516632474996 73.5076292690142 76.00577726054674 [1116.72121466 1111.68355796 1107.09933253 1106.43204635 1105.15276614
 1103.04008526 1100.26094059 1096.84704972 1093.23134376 1089.98770904
 1087.354029   1085.03306197]

Got an mse at 0.0025 in round 400 and stopped training

69 69 12
Wall time: 16 s
1040 3.3871576626016524 46.962453272183886 50.86184730768627 [1095.0340069  1088.58096215 1082.57807731 1074.5839888  1079.72267881
 1083.09786844 1085.4084366  1086.87392495 1087.23730583 1086.49750116
 1084.07800948 1080.55622314]

Got an mse at 0.0025 in round 259 and stopped training


Got an mse at 0.0025 in round 431 and stopped training

69 69 12
Wall time: 15.3 s
1845 5.262355038165737 47.964367175848984 48.733102743906315 [1809.903322   1808.59425616 1808.32360686 1812.8657994  1813.55865077
 1812.56206686 1811.29287709 1809.6136544  1807.52023002 1805.03991632
 1803.70388648 1802.21664511]

Got an mse at 0.0025 in round 346 and stopped training

69 69 12
Wall time: 13.7 s
1850 7.0119213479478795 108.70651799653511 100.35216526527672 [1988.11082978 1983.5182972  1980.44541229 1980.80603246 1980.78022483
 1980.17236197 1978.39216499 1974.50259191 1967.0884602  1956.24952126
 1945.04283424 1936.92990089]

Got an mse at 0.0025 in round 375 and stopped training

69 69 12
Wall time: 14.2 s
1851 6.16479637828504 110.83737924266755 110.06188167820245 [1994.58948668 1990.53548635 1986.76398358 1977.83591919 1967.66556484
 1956.93627628 1946.87244201 1938.20204851 1933.80463133 1932.34240312
 1932.99358238 1933.04454398]

Got an mse at 0.0025 in round 316 and stopped tr

69 69 12
Wall time: 15.3 s
2125 7.975304515485945 205.22418282989588 205.95118991153657 [2481.11948857 2474.28464818 2472.86327341 2469.83930372 2466.83187077
 2463.97825182 2461.40904626 2459.41130276 2457.79933494 2456.46930808
 2455.36285177 2454.42825156]
69 69 12
Wall time: 17 s
2126 6.482231815501955 198.6658715217311 198.32326707401162 [2286.34152625 2283.55321171 2281.5531663  2280.94848801 2283.89922542
 2286.89834584 2288.60805758 2289.7659044  2290.55654781 2291.20786142
 2292.04794861 2293.20904573]
69 69 12
Wall time: 16.4 s
2127 6.176344049795446 111.26747941898438 108.15637597192004 [2867.19994413 2872.792475   2877.1319346  2883.70250425 2884.82259174
 2885.30227892 2885.29480374 2884.87013677 2884.06907323 2882.91577157
 2881.43276237 2879.6628185 ]

Got an mse at 0.0025 in round 248 and stopped training

69 69 12
Wall time: 11.4 s
2128 8.688550304127933 158.28820886685483 169.21141387809436 [2472.23441459 2462.57206757 2455.21605576 2451.32323636 2454.36348787
 2458.7


Got an mse at 0.0025 in round 200 and stopped training

69 69 12
Wall time: 10.8 s
2184 8.736307473019222 111.43484513530227 110.62410681933466 [2153.88391033 2137.73653209 2125.90902596 2120.40321668 2116.53532747
 2113.70296659 2111.23832517 2108.15210481 2103.25686721 2095.40077795
 2084.84449742 2074.28921563]
69 69 12
Wall time: 17.3 s
2210 5.3860799028162365 155.4941518665783 150.54604509715466 [2891.8103494  2894.56916488 2891.95768675 2887.06349137 2882.63505727
 2877.92792484 2872.5411316  2866.30773953 2859.32769024 2851.85150738
 2844.26300296 2836.98151978]

Got an mse at 0.0025 in round 240 and stopped training

69 69 12
Wall time: 11.3 s
2215 7.945092690118122 143.4415315027758 148.17666702663314 [2369.11354884 2372.0899219  2375.67557814 2397.58167414 2406.67857138
 2414.67578402 2421.92178857 2427.81036783 2432.56781925 2436.1568222
 2439.33112581 2442.40559645]

Got an mse at 0.0025 in round 164 and stopped training

69 69 12
Wall time: 9.59 s
2301 6.586163254745419 9


Got an mse at 0.0025 in round 424 and stopped training

69 69 12
Wall time: 15.2 s
2864 4.061740554846209 23.414396905378442 23.758860635063982 [1288.95882175 1292.35326273 1294.99137157 1293.13461323 1290.84564178
 1287.92446089 1283.92813771 1278.1424986  1269.44525304 1257.55072261
 1244.63959493 1232.98873763]

Got an mse at 0.0025 in round 277 and stopped training

69 69 12
Wall time: 12.1 s
2891 3.3156993042492795 44.675086279319714 45.33073787302712 [1213.04987728 1210.4208921  1211.28392592 1214.48332898 1215.02630333
 1203.92484244 1191.0028323  1178.85776535 1166.80219605 1154.47663632
 1142.91868883 1132.36705581]
69 69 12
Wall time: 17.2 s
2893 3.9858434896329276 14.939660787996655 15.478386251926766 [1234.19751037 1234.610501   1233.80508605 1230.32549925 1226.21541043
 1221.36197963 1215.52186199 1210.22238971 1206.70150659 1203.41845394
 1200.84733133 1199.02382247]

Got an mse at 0.0025 in round 339 and stopped training

69 69 12
Wall time: 14.1 s
2895 3.85172420837315

...


Got an mse at 0.0025 in round 62 and stopped training

69 69 12
Wall time: 6.11 s
92507 6.294944497651752 168.79881574076816 177.91144876313015 [1467.5279211  1470.61294603 1474.65242768 1474.42216116 1471.88192682
 1468.0685026  1458.86063361 1445.98166589 1436.82515644 1430.27537841
 1422.78646875 1415.14383891]

Got an mse at 0.0025 in round 286 and stopped training

69 69 12
Wall time: 10.5 s
92602 7.273776854727547 75.66138390100224 78.85977620280008 [2484.01010909 2488.71925129 2489.53246657 2490.63183928 2489.76842166
 2481.72814432 2471.71785179 2460.24543681 2447.84041894 2435.80636567
 2422.744311   2409.05049472]

Got an mse at 0.0025 in round 295 and stopped training

69 69 12
Wall time: 10.6 s
92603 4.489682148083726 45.60382658834349 42.262635003442426 [2879.66375982 2875.82813239 2873.82260025 2882.76815373 2891.8562051
 2900.68808457 2908.43382001 2917.17420791 2927.00528814 2934.07170241
 2939.20923888 2941.95499851]

Got an mse at 0.0025 in round 168 and stopped trai


Got an mse at 0.0025 in round 100 and stopped training

69 69 12
Wall time: 7.06 s
93030 7.0627386464696515 96.6841959282336 96.35449600600337 [1884.75387076 1896.90175512 1908.00637981 1902.49427644 1894.96101731
 1885.0440024  1872.14688684 1855.53107256 1833.69065908 1809.53620615
 1790.14226629 1776.44552133]

Got an mse at 0.0025 in round 315 and stopped training

69 69 12
Wall time: 11.1 s
93304 1.8850939923173335 22.271422847777576 22.27906691738521 [880.98968602 879.66904944 876.69046091 870.57139141 863.62068937
 857.54201771 854.18383066 852.80712549 851.57736093 850.2623359
 848.82717723 847.36010628]

Got an mse at 0.0025 in round 349 and stopped training

69 69 12
Wall time: 11.6 s
93305 1.8945795004954151 39.10408719312031 39.20953896883837 [788.19662362 789.44424619 790.99644474 791.78086901 791.85307451
 791.58875386 791.16322713 790.68210297 790.24933482 790.21245303
 790.43008196 790.67032705]

Got an mse at 0.0025 in round 314 and stopped training

69 69 12
Wall tim


Got an mse at 0.0025 in round 192 and stopped training

69 69 12
Wall time: 8.73 s
94530 8.834548952355739 89.57722514420563 94.23527182144566 [2482.91860221 2496.98365454 2506.87928468 2507.38050834 2508.44087264
 2505.9819982  2499.36384683 2489.04402258 2474.88135936 2462.01611046
 2453.16460307 2448.18602789]

Got an mse at 0.0025 in round 52 and stopped training

69 69 12
Wall time: 5.83 s
94538 9.322545503701456 130.89477335121288 130.05653558236818 [2476.83852763 2476.16927936 2470.99392693 2461.05306109 2449.61026785
 2436.50651861 2421.8095812  2405.79300722 2388.52939315 2371.45488085
 2357.79052594 2346.69332715]

Got an mse at 0.0025 in round 137 and stopped training

69 69 12
Wall time: 7.6 s
94541 12.279886798345157 110.4850589985647 113.34120915862826 [2356.47543296 2352.75205083 2350.09662243 2348.341306   2345.93822925
 2342.67773962 2337.39517555 2330.87388262 2326.4266961  2322.57032641
 2318.91855435 2315.68932426]

Got an mse at 0.0025 in round 196 and stopped tra


Got an mse at 0.0025 in round 196 and stopped training

69 69 12
Wall time: 9.05 s
95678 6.763804949896472 104.04072941818147 104.50727107312463 [1657.3711182  1656.11285331 1654.77557617 1654.78559936 1653.97210297
 1652.41044089 1650.3001732  1648.09000028 1647.27015468 1644.69981679
 1642.41086796 1640.45752864]

Got an mse at 0.0025 in round 62 and stopped training

69 69 12
Wall time: 6.22 s
95816 10.856638601577387 196.5198554655073 196.03601141851254 [1462.49311536 1466.51708496 1466.01054059 1450.6395473  1433.44220736
 1413.37007009 1391.6256572  1375.43535365 1368.5861452  1365.41335305
 1362.23432797 1359.09788007]

Got an mse at 0.0025 in round 290 and stopped training

69 69 12
Wall time: 10.8 s
95826 9.197443558574804 56.61755797688138 57.71881311143895 [1433.93871506 1431.66042547 1430.93942207 1428.56626704 1425.67263378
 1422.27987281 1417.80141338 1411.62019868 1404.64642051 1397.38412217
 1389.95327003 1382.34457515]

Got an mse at 0.0025 in round 92 and stopped tra


Got an mse at 0.0025 in round 218 and stopped training

69 69 12
Wall time: 8.41 s
98101 13.073541376371486 162.68801457531825 156.9311301608241 [2558.33587309 2539.23487834 2522.3676273  2516.40725555 2506.24401116
 2493.85778676 2481.60470315 2470.13797145 2459.39814228 2449.16032807
 2439.2100851  2429.39078141]

Got an mse at 0.0025 in round 223 and stopped training

69 69 12
Wall time: 8.43 s
98102 6.055329934637931 48.058152815690796 51.49174602230362 [1991.84925201 1985.54245359 1982.86956973 1982.33688629 1983.11759429
 1984.6542049  1987.28172009 1987.22013306 1989.19364511 1993.54220674
 1999.14146083 2002.30630644]

Got an mse at 0.0025 in round 190 and stopped training

69 69 12
Wall time: 7.93 s
98103 7.195653412390631 55.080028223805186 54.46893017620129 [1935.9559736  1935.54516865 1934.26448123 1931.7867482  1928.45013073
 1924.26231234 1919.48946183 1914.43812738 1909.3634152  1904.37759389
 1899.45532576 1894.48096397]

Got an mse at 0.0025 in round 160 and stopped t

In [14]:
print(len(score_dict["RMSE_pure"]), sum(score_dict["RMSE_test"])/len(score_dict["RMSE_test"]),
      sum(score_dict["RMSE_pure"])/len(score_dict["RMSE_pure"]));

1301 62.23074739431485 62.582817668335196


In [15]:
test = pd.DataFrame(score_dict);
test.head()

,zip,RMSE_train,RMSE_test,RMSE_pure,forecast
0,1013,3.675370,67.650902,68.558858,"[1094.2462559674186, 1088.9382427771252, 1084...."
1,1020,3.737517,73.507629,76.005777,"[1116.7212146567865, 1111.6835579559665, 1107...."
2,1040,3.387158,46.962453,50.861847,"[1095.0340069003805, 1088.5809621451224, 1082...."
3,1085,3.769593,55.508633,55.073420,"[1151.7875662977056, 1144.7039507931427, 1138...."
4,1104,3.569507,72.240296,71.541655,"[1116.4557472499403, 1111.9595782639135, 1105...."


In [16]:
test.to_csv('NN_feature_window_3_overoverfit.csv', index = False)